In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv(r"C:\Users\paula\OneDrive\Documents\Financial Modelling Package\raw_data.csv", sep = ";")

The corrected SVI model fit with the maturity

In [2]:
import numpy as np
from scipy.optimize import minimize

class SVI_model:
    def __init__(self, initial_params=None):
        if initial_params is None:
            self.params = {"a": 0.04, "b": 0.1, "rho": -0.3, "m": 0.0, "sigma": 0.2}
        else:
            self.params = initial_params

    def svi_implied_vol(self, k, T, a, b, rho, m, sigma):
        # Compute total variance w(k):
        w_k = a + b * (rho * (k - m) + np.sqrt((k - m)**2 + sigma**2))
        # Return implied volatility:
        return np.sqrt(w_k / T)

    def fit(self, log_moneyness, maturity, market_ivol):
        """
        log_moneyness, market_ivol: numpy arrays of same length
        maturity: float (single T) or array of T if you handle multiple
        """
        def objective(pars):
            a, b, rho, m, sigma = pars
            predicted_vols = self.svi_implied_vol(log_moneyness, maturity, a, b, rho, m, sigma)
            return np.sum((predicted_vols - market_ivol) ** 2)

        initial_guess = list(self.params.values())
        bounds = [(0, None), (0, None), (-1, 1), (None, None), (1e-3, None)]
        result = minimize(objective, initial_guess, bounds=bounds)
        if result.success:
            self.params = dict(zip(["a", "b", "rho", "m", "sigma"], result.x))
        else:
            raise ValueError("SVI calibration failed.")

    def predict_ivol(self, k, T):
        a, b, rho, m, sigma = self.params.values()
        return self.svi_implied_vol(k, T, a, b, rho, m, sigma)


now let's fit the thing

In [1]:
from financial_modelling.data_pre_processing.IVPreprocessor import IVPreprocessor

filtered_data = IVPreprocessor(data).preprocess((0.90, 1.10),(0.90, 1.10))

model = SVI_model()

for i, value in enumerate(filtered_data["Residual_Maturity"].unique()):
    # 3) Fit
    model.fit(
        log_moneyness=filtered_data["Log_Moneyness"].values,
        maturity= value,
        market_ivol=filtered_data["Implied_Volatility"].astype(float).values
    )

    # 4) Check calibrated parameters
    print("Calibrated params:", model.params)

    # 5) Predict implied vols on the same points (just for demonstration)
    pred_iv = model.predict_ivol(df["k"].values, df["T"].values)
    print("Predicted implied vols:", pred_iv)

: 